In [ ]:
import os
import numpy as np
import librosa
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import img_to_array

# Function to load data
def load_data(data_path, target_shape=(128, 128)):
    X = []
    y = []
    genres = os.listdir(data_path)

    for genre in genres:
        genre_folder = os.path.join(data_path, genre)
        if os.path.isdir(genre_folder):
            for filename in os.listdir(genre_folder):
                if filename.endswith('.wav'):
                    file_path = os.path.join(genre_folder, filename)
                    print(f"Loading file: {file_path}")  # Debugging line
                    signal, sr = librosa.load(file_path, sr=None)
                    mel_spectrogram = librosa.feature.melspectrogram(y=signal, sr=sr)
                    mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)
                    
                    # Resize or pad the mel spectrogram to the target shape
                    mel_spectrogram_resized = np.zeros(target_shape)
                    min_shape = min(mel_spectrogram.shape[1], target_shape[1])
                    mel_spectrogram_resized[:, :min_shape] = mel_spectrogram[:, :min_shape]
                    
                    X.append(mel_spectrogram_resized[..., np.newaxis])  # Add channel dimension
                    y.append(genres.index(genre))  # Encode genre as a number

    print(f"Loaded {len(X)} samples from {len(genres)} genres.")
    return np.array(X), np.array(y)

# Load data
data_path = 'E:\\Coding\\Python\\.Projects\\MusicGenreClassification\\data\\training'
X, y = load_data(data_path)

if len(X) == 0 or len(y) == 0:
    print("No data loaded. Check your data path and files.")
else:
    # Normalize the features
    X = (X - np.mean(X)) / np.std(X)

    # Split the dataset
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

    # Check shapes of training and validation data
    print(f"Training data shape: {X_train.shape}, Validation data shape: {X_val.shape}")

    # Build the model
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(X.shape[1], X.shape[2], 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(len(np.unique(y)), activation='softmax'))

    # Compile the model
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    # Train the model
    print("Starting model training...")
    model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32, verbose=1)
    print("Model training completed.")

    # Save the model if needed
    model.save('music_genre_model.h5')

    # Function to predict classes
    def predict_genre(file_path):
        signal, sr = librosa.load(file_path, sr=None)
        mel_spectrogram = librosa.feature.melspectrogram(y=signal, sr=sr)
        mel_spectrogram = librosa.power_to_db(mel_spectrogram, ref=np.max)

        # Resize or pad the mel spectrogram to the target shape
        mel_spectrogram_resized = np.zeros((128, 128))
        min_shape = min(mel_spectrogram.shape[1], 128)
        mel_spectrogram_resized[:, :min_shape] = mel_spectrogram[:, :min_shape]

        mel_spectrogram_resized = mel_spectrogram_resized[..., np.newaxis]  # Add channel dimension
        mel_spectrogram_resized = (mel_spectrogram_resized - np.mean(mel_spectrogram_resized)) / np.std(mel_spectrogram_resized)
        mel_spectrogram_resized = np.expand_dims(mel_spectrogram_resized, axis=0)  # Add batch dimension

        prediction = model.predict(mel_spectrogram_resized)
        predicted_class = np.argmax(prediction)
        return predicted_class

    # Example of predicting genre for test data
    test_data_path = 'E:\\Coding\\Python\\.Projects\\MusicGenreClassification\\data\\testing'
    test_files = os.listdir(test_data_path)

    print(f"Found {len(test_files)} test files.")

    if len(test_files) == 0:
        print("No test files found. Check your test data path.")
    else:
        for test_file in test_files:
            if test_file.endswith('.wav'):
                file_path = os.path.join(test_data_path, test_file)
                print(f"Predicting genre for: {file_path}")  # Debugging line
                predicted_class = predict_genre(file_path)
                print(f"File: {test_file}, Predicted Class: {predicted_class}")
            else:
                print(f"Skipping non-audio file: {test_file}")
